# closure가 variable scope와 상호 작용하는 방법을 알자

리스트를 정렬할 때 특정 그룹의 값들을 먼저 오도록 우선순위를 매기는 등의 패턴은 사용자 인터페이스를 표현하거나, 다른 것보다 중요한 메시지나 예외 이벤트를 먼저 보여줘야 할 때 유용하다.

## 이렇게 만드는 일반적인 방법

__리스트의 sort method에 헬퍼 함수를 key 인수로 넘기는 것이다.__ 핼퍼의 반환 값은 리스트에 있는 각 아이템을 정렬하는 값으로 사용된다. 핼퍼는 주어진 아이템이 중요한 그룹에 있는지 확인하고 그에 따라 정렬 키를 다르게 할 수 있다.

In [1]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

함수는 간단한 입력값에 사용한다.

In [3]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


## 함수가 예상대로 동작한 이유 3가지

### 1. Python은 closure를 지원한다.

__closure: 자신이 정의된 범위에 있는 변수를 참조하는 함수__

덕분에 helper 함수가 sort_priority의 group 인수에 접근할 수 있다.

### 2. 함수는 Python에서 first-class object다.

__함수를 직접 참조하고, 변수에 할당하고, 다른 함수의 인수로 전달하고, 표현식과 if문 등에서 비교할 수 있다.__

따라서 sort method에서 closure 함수를 key 인수로 받을 수 있다.

### 3. Python에는 tuple을 비교하는 특정한 규칙이 있다.

index 0으로 아이템을 비교하고 그 다음으로 순차적으로 진행한다. __helper closure의 반환 값이 정렬 순서를 분리된 두 그룹으로 나뉘게 한 건 이 규칙 때문이다.__

### 우선순위가 높은 아이템 발견 여부를 반환해서 사용자 인터페이스 코드가 그에 따라 동작하게 하는 법

이미 각 숫자가 어느 그룹에 포함되어 있는지 판별하는 closure 함수가 있다. 

아래 예는 우선순위가 높은 아이템을 발견했을 때 flag를 뒤집는 데도 closure를 활용했다.

In [8]:
def sort_priority2(numbers, group):
    found = False
    def helper(x):
        if x in group:
            found = True # 간단해 보임
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

In [9]:
found = sort_priority2(numbers, group)
print('Found:', found)
print(numbers)

Found: False
[2, 3, 5, 7, 1, 4, 6, 8]


__표현식에서 변수를 참조할 때 Python Interpreter는 참조를 해결하려고 다음과 같은 순서로 scope를 탐색한다.__

1. 현재 함수의 scope
2. (현재 scope를 담고 있는 다른 함수 같은) 감싸고 있는 scope
3. 코드를 포함하고 있는 모듈의 scope(전역 scope)
4. (len이나 str 같은 함수를 담고 있는) 내장 scope

이중 어느 scope에도 참조한 이름으로 된 변수가 정의되어 있지 않으면 NameError 예외가 일어난다.

## 변수에 값을 할당할 때 동작 방식

변수가 이미 현재 scope에 정의되어 있다면 새로운 값을 얻는다. Python은 변수가 현재 scope에 존재하지 않으면 변수 정의로 취급한다. 새로 정의되는 변수의 scope는 그 할당을 포함하고 있는 함수가 된다.

이 할당 동작은 sort_priority2 함수의 반환 값이 잘못된 이유를 설명해준다. found 변수는 helper closure에서 True로 할당된다. closure 할당은 sort_priority2에서 일어나는 할당이 아닌 helper 안에서 일어나는 새 변수 정의로 처리된다.

In [10]:
def sort_priority2(numbers, group):
    found = False         # scope: 'sort_priority'
    def helper(x):
        if x in group:
            found = True  # scope: 'helper' -- bad!!
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

## scoping bug

언어 설계자가 의도한 결과다. __이 동작은 함수의 지역 변수가 자신을 포함하는 모듈을 오염시키는 문제를 막아준다.__ 그렇지 않았다면 함수 안에서 일어나는 모든 할당이 전역 모듈 스코프에 쓰레기를 넣는 결과로 이어졌을 것이다. 그렇게 되면 불필요한 할당에 그치지 않고 결과로 만들어지느 전역 변수들의 상호 작용으로 알기 힘든 버그가 생긴다.

## 데이터 읽어오기

nonlocal문은 특정 변수 이름에 할당할 때 scope 탐색이 일어나야 함을 나타낸다. 유일한 제약은 nonlocal이 모듈 수준 scope까지는 탐색할 수 없다는 점이다.

In [11]:
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

nonlocal문은 closure에서 데이터를 다른 scope에 할당하는 시점을 알아보기 쉽게 해준다. nonlocal 문은 변수 할당이 모듈 scope에 직접 들어가게 하는 global 문을 보완한다.

하지만 전역 변수의 anti-pattern과 마찬가지로 간단한 함수 이외에 nonlocal을 사용하지 않도록 주의해야 한다. nonlocal의 부작용은 알아내기 상당히 어렵다. 특히 nonlocal 문과 관련 변수에 대한 할당이 멀리 떨어진 긴 함수에서는 이해하기가 더욱 어렵다.

nonlocal 사용 시 복잡해지기 시작하면 헬퍼 클래스로 상태를 감싸는 방법을 이용하는 게 낫다.

In [12]:
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
        
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)
    
sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True

## Python2의 scope

Python2는 nonlocal 키워드를 지원하지 않는다.

다음은 일반적인 Python 표현 방식이다.

In [13]:
# Python2
def sort_priority(numbers, group):
    found = [False]
    def helper(x):
        if x in group:
            found[0] = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found[0]

Python은 현재 값을 알아내려고 found 변수가 어디서 참조되었는지 상위 scope로 탐색해나간다.  
__트릭은 found 값이 mutable list라는 점이다.__  
closure에서 일단 found를 받아온 후에 내부 scope에서 found의 상태를 바꿔서 데이터를 보낼 수 있다는 의미다.

이 방법은 scope를 탐색하는 데 사용되는 변수가 dictionary나 set 혹은 사용자 정의 클래스의 인스턴스일 때도 적용된다.